# 🔍 Reddit Real Estate Post Analysis

### Objective:
Analyze post titles from r/RealEstate to uncover commonly used keywords and patterns in housing-related discussions.

This notebook includes:
- Connecting to PostgreSQL
- Extracting most common words from post titles
- Categorizing words by length
- Preparing data for Looker Studio visualizations


In [14]:
!pip install psycopg2-binary




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set up database connection
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)
print("✅ Connected to PostgreSQL.")


✅ Connected to PostgreSQL.


In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)


### 🧠 Query 3: Words Frequently Mentioned with "Buy" or "Sell" (Diagnostic Query)

This query focuses on post titles that mention the keywords "buy" or "sell." It helps us understand the context in which these words are used. 
Are users asking questions, giving advice, or sharing listings? This diagnostic query complements our descriptive word frequency analysis.


In [2]:
query_3 = """
SELECT 
  word,
  COUNT(*) AS word_count
FROM (
    SELECT unnest(string_to_array(lower(title), ' ')) AS word
    FROM sql_project.reddit_raw
) AS words
WHERE word IN ('buy', 'buyer', 'buying', 'sell', 'seller', 'selling', 'sold')
GROUP BY word
ORDER BY word_count DESC
LIMIT 50;
"""

df_buy_sell = pd.read_sql(query_3, engine)
df_buy_sell


,word,word_count
0,selling,3
1,buyer,2
2,buy,1
3,sold,1
4,buying,1


## 🧠 Query 1: Most Common Words in Reddit Post Titles

This query splits each post title into individual words, lowercases them, and counts how often each word appears. Common keywords reveal user concerns and themes.


In [ ]:
query_1 = """
SELECT word, COUNT(*) AS word_count
FROM (
    SELECT unnest(string_to_array(lower(title), ' ')) AS word
    FROM sql_project.reddit_raw
) AS words
GROUP BY word
ORDER BY word_count DESC
LIMIT 50;
"""

df_word_counts = pd.read_sql(query_1, engine)
df_word_counts


,word,word_count
0,to,8
1,is,5
2,from,4
3,my,4
4,a,3
5,if,3
6,what,3
7,i,3
8,advice,3
9,selling,3


## 🔤 Query 2: Categorize Words by Length

To better understand word patterns, we categorize them into:
- Short (≤4 letters)
- Medium (5–7 letters)
- Long (8+ letters)


In [ ]:
query_2 = """
SELECT 
  CASE
    WHEN length(word) <= 4 THEN 'Short'
    WHEN length(word) BETWEEN 5 AND 7 THEN 'Medium'
    ELSE 'Long'
  END AS length_category,
  word,
  COUNT(*) AS word_count
FROM (
    SELECT unnest(string_to_array(lower(title), ' ')) AS word
    FROM sql_project.reddit_raw
) AS words
GROUP BY length_category, word
ORDER BY word_count DESC
LIMIT 50;
"""

df_length_category = pd.read_sql(query_2, engine)
df_length_category


,length_category,word,word_count
0,Short,to,8
1,Short,is,5
2,Short,from,4
3,Short,my,4
4,Short,a,3
5,Short,if,3
6,Medium,advice,3
7,Short,what,3
8,Medium,selling,3
9,Short,i,3


🔍 Business Question 3 (Diagnostic Query)
What words are most frequently used in Reddit post titles that mention “buy” or “sell”?

These posts are likely to reflect transactional discussions, giving insight into what users care about most when buying or selling real estate. This diagnostic query helps explain why keywords like "advice," "listing," or "offer" appear frequently in our overall word frequency analysis.